In [1]:
import pandas as pd
from pandas.plotting import lag_plot
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller
import scipy
import datetime as dt
import numpy as np
from pandas import concat
from statsmodels.tsa.ar_model import AR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from math import sqrt
from pmdarima.arima import auto_arima
from statsmodels.tsa.arima_model import ARIMA
from matplotlib import pyplot as plt
%matplotlib inline  

from pandas.plotting import autocorrelation_plot

# Tripoli Weather Data

- NASA/POWER SRB/FLASHFlux/MERRA2/GEOS 5.12.4 (FP-IT) 0.5 x 0.5 Degree Daily Averaged Data 
- Dates (month/day/year): 01/01/2015 through 07/19/2021 
- Location: Latitude  32.8994   Longitude 13.2026 
- Elevation from MERRA-2: Average for 1/2x1/2 degree lat/lon region = 104.28 meters   Site = na 
- Climate zone: na (reference Briggs et al: http://www.energycodes.gov) 
- Value for missing model data cannot be computed or out of model availability range: -999 


# Italy_other_Weather_Data
Parameter(s): 
- WS10M_MIN MERRA2 1/2x1/2 Minimum Wind Speed at 10 Meters (m/s) 
- WS50M_RANGE MERRA2 1/2x1/2 Wind Speed Range at 50 Meters (m/s) 
- T2M_RANGE MERRA2 1/2x1/2 Temperature Range at 2 Meters (C) 
- WS10M_RANGE MERRA2 1/2x1/2 Wind Speed Range at 10 Meters (m/s) 
- TS MERRA2 1/2x1/2 Earth Skin Temperature (C) 
- T2MWET MERRA2 1/2x1/2 Wet Bulb Temperature at 2 Meters (C) 
- T2M MERRA2 1/2x1/2 Temperature at 2 Meters (C) 
- WS10M_MAX MERRA2 1/2x1/2 Maximum Wind Speed at 10 Meters (m/s) 
- WS50M_MAX MERRA2 1/2x1/2 Maximum Wind Speed at 50 Meters (m/s) 
- T2M_MAX MERRA2 1/2x1/2 Maximum Temperature at 2 Meters (C) 
- WS50M MERRA2 1/2x1/2 Wind Speed at 50 Meters (m/s) 
- WS10M MERRA2 1/2x1/2 Wind Speed at 10 Meters (m/s) 
- T2MDEW MERRA2 1/2x1/2 Dew/Frost Point at 2 Meters (C) 
- WS50M_MIN MERRA2 1/2x1/2 Minimum Wind Speed at 50 Meters (m/s) 
- T2M_MIN MERRA2 1/2x1/2 Minimum Temperature at 2 Meters (C) 
-END HEADER-


In [36]:
weather_data = pd.read_csv("Tripoli_Weahter_Data.csv", sep = ",", dayfirst=True)

In [37]:
weather_data.head()


,LAT,LON,YEAR,MO,DY,T2M_RANGE,TS,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M,WS50M_RANGE,WS10M_RANGE,WS50M_MIN,WS10M_MIN,WS50M_MAX,WS10M_MAX,WS50M,WS10M
0,32.89941,13.20261,2015,1,1,5.94,11.20,5.05,5.06,11.70,5.75,8.74,7.53,6.73,7.51,6.17,15.04,12.90,10.59,8.77
1,32.89941,13.20261,2015,1,2,2.42,13.34,7.09,7.10,13.74,11.31,12.14,3.40,2.80,7.84,5.76,11.24,8.57,9.42,7.46
2,32.89941,13.20261,2015,1,3,3.90,13.39,8.12,8.12,14.58,10.68,12.62,1.66,1.34,7.24,5.35,8.90,6.69,8.13,6.19
3,32.89941,13.20261,2015,1,4,7.52,12.98,5.82,5.84,15.92,8.40,11.94,4.28,3.92,8.19,6.49,12.47,10.41,10.89,8.92
4,32.89941,13.20261,2015,1,5,6.39,13.53,8.51,8.51,16.40,10.00,12.98,3.23,3.45,8.23,5.98,11.46,9.43,9.87,7.78


In [38]:
weather_data["Date"] = pd.to_datetime((weather_data.YEAR*10000+weather_data.MO*100+weather_data.DY).apply(str),format='%Y%m%d')
weather_data = weather_data.set_index("Date")
weather_data = weather_data.drop(["LON","LAT","MO","YEAR","DY"], axis = 1)
weather_data.index = pd.to_datetime(weather_data.index)

In [ ]:
weather_data